In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)


try:
    df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='latin-1')
except UnicodeDecodeError:
    # If latin-1 fails, try another common encoding
    df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='cp1252')


print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows of the dataset:")
display(df.head())

# Check for missing values
print("\nMissing values in each column:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Check data types
print("\nData types:")
print(df.dtypes)

Dataset shape: (180519, 53)

First 5 rows of the dataset:


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.25,314.64,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.00,2,Fitness,18.25,-66.04,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.11,0.04,180517,327.75,0.29,1,327.75,314.64,91.25,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.09,311.36,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.00,2,Fitness,18.28,-66.04,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.39,0.05,179254,327.75,-0.80,1,327.75,311.36,-249.09,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.78,309.72,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.00,2,Fitness,37.29,-121.88,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.03,0.06,179253,327.75,-0.80,1,327.75,309.72,-247.78,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.86,304.81,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.00,2,Fitness,34.13,-118.29,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.94,0.07,179252,327.75,0.08,1,327.75,304.81,22.86,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.21,298.25,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.00,2,Fitness,18.25,-66.04,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.50,0.09,179251,327.75,0.45,1,327.75,298.25,134.21,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class



Missing values in each column:
Customer Lname              8
Customer Zipcode            3
Order Zipcode          155679
Product Description    180519
dtype: int64

Data types:
Type                              object
Days for shipping (real)           int64
Days for shipment (scheduled)      int64
Benefit per order                float64
Sales per customer               float64
Delivery Status                   object
Late_delivery_risk                 int64
Category Id                        int64
Category Name                     object
Customer City                     object
Customer Country                  object
Customer Email                    object
Customer Fname                    object
Customer Id                        int64
Customer Lname                    object
Customer Password                 object
Customer Segment                  object
Customer State                    object
Customer Street                   object
Customer Zipcode                 float64
De

In [15]:
# Convert date columns to datetime
date_columns = ['order date (DateOrders)', 'shipping date (DateOrders)']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Convert numeric columns that are stored as strings
numeric_cols = [
    'Benefit per order', 'Sales per customer', 'Order Item Discount',
    'Order Item Discount Rate', 'Order Item Product Price', 'Order Item Profit Ratio',
    'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Profit Per Order', 'Product Price'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape: {df.shape}")

# Calculate shipping delay (difference between real and scheduled days)
df['Shipping Delay'] = df['Days for shipping (real)'] - df['Days for shipment (scheduled)']


Number of duplicate rows: 0


In [16]:
print("\n\n# Exploratory Data Analysis")
print("============================")

# Basic statistics
print("\nBasic statistics for numeric columns:")
print(df.describe())

# Distribution of customer segments
plt.figure(figsize=(10, 6))
segment_counts = df['Customer Segment'].value_counts()
print("\nCustomer Segment Distribution:")
print(segment_counts)

# Create an interactive pie chart for customer segments
fig = px.pie(
    names=segment_counts.index,
    values=segment_counts.values,
    title='Distribution of Customer Segments',
    hole=0.4,
)
fig.update_traces(textinfo='percent+label')
fig.show()

# Distribution of order status
plt.figure(figsize=(10, 6))
status_counts = df['Order Status'].value_counts()
print("\nOrder Status Distribution:")
print(status_counts)

# Create an interactive bar chart for order status
fig = px.bar(
    x=status_counts.index,
    y=status_counts.values,
    title='Distribution of Order Status',
    labels={'x': 'Order Status', 'y': 'Count'},
    color=status_counts.values,
    color_continuous_scale='Viridis',
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# Distribution of shipping modes
plt.figure(figsize=(10, 6))
shipping_counts = df['Shipping Mode'].value_counts()
print("\nShipping Mode Distribution:")
print(shipping_counts)

# Create an interactive bar chart for shipping modes
fig = px.bar(
    x=shipping_counts.index,
    y=shipping_counts.values,
    title='Distribution of Shipping Modes',
    labels={'x': 'Shipping Mode', 'y': 'Count'},
    color=shipping_counts.values,
    color_continuous_scale='Viridis',
)
fig.show()




# Exploratory Data Analysis

Basic statistics for numeric columns:
       Days for shipping (real)  Days for shipment (scheduled)  \
count                 180519.00                      180519.00   
mean                       3.50                           2.93   
min                        0.00                           0.00   
25%                        2.00                           2.00   
50%                        3.00                           4.00   
75%                        5.00                           4.00   
max                        6.00                           4.00   
std                        1.62                           1.37   

       Benefit per order  Sales per customer  Late_delivery_risk  Category Id  \
count          180519.00           180519.00           180519.00    180519.00   
mean               21.97              183.11                0.55        31.85   
min             -4274.98                7.49                0.00         2.00   
25%         


Order Status Distribution:
Order Status
COMPLETE           59491
PENDING_PAYMENT    39832
PROCESSING         21902
PENDING            20227
CLOSED             19616
ON_HOLD             9804
SUSPECTED_FRAUD     4062
CANCELED            3692
PAYMENT_REVIEW      1893
Name: count, dtype: int64



Shipping Mode Distribution:
Shipping Mode
Standard Class    107752
Second Class       35216
First Class        27814
Same Day            9737
Name: count, dtype: int64


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [17]:
# Distribution of delivery status
plt.figure(figsize=(10, 6))
delivery_counts = df['Delivery Status'].value_counts()
print("\nDelivery Status Distribution:")
print(delivery_counts)

# Create an interactive pie chart for delivery status
fig = px.pie(
    names=delivery_counts.index,
    values=delivery_counts.values,
    title='Distribution of Delivery Status',
)
fig.update_traces(textinfo='percent+label')
fig.show()


Delivery Status Distribution:
Delivery Status
Late delivery        98977
Advance shipping     41592
Shipping on time     32196
Shipping canceled     7754
Name: count, dtype: int64


<Figure size 1000x600 with 0 Axes>

In [18]:
print("\n\n# Customer Analysis")
print("===================")

# Group by customer ID and calculate total order quantity and profit
customer_analysis = df.groupby('Order Customer Id').agg({
    'Order Item Quantity': 'sum',
    'Order Profit Per Order': 'sum',
    'Customer Segment': 'first',
    'Customer Country': 'first',
    'Customer City': 'first',
    'Customer Fname': 'first',
    'Customer Lname': 'first'
}).reset_index()

customer_analysis['Customer Name'] = customer_analysis['Customer Fname'] + ' ' + customer_analysis['Customer Lname']

# Top 20 customers by order quantity
top_quantity = customer_analysis.sort_values('Order Item Quantity', ascending=False).head(20)
print("\nTop 20 Customers by Order Quantity:")
print(top_quantity[['Customer Name', 'Customer Segment', 'Customer Country', 'Order Item Quantity']])

# Create an interactive bar chart for top customers by quantity
fig = px.bar(
    top_quantity,
    x='Customer Name',
    y='Order Item Quantity',
    color='Customer Segment',
    title='Top 20 Customers by Order Quantity',
    labels={'Order Item Quantity': 'Total Quantity', 'Customer Name': 'Customer'},
    hover_data=['Customer Country', 'Customer City']
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# Top 20 customers by profit
top_profit = customer_analysis.sort_values('Order Profit Per Order', ascending=False).head(20)
print("\nTop 20 Customers by Profit:")
print(top_profit[['Customer Name', 'Customer Segment', 'Customer Country', 'Order Profit Per Order']])

# Create an interactive bar chart for top customers by profit
fig = px.bar(
    top_profit,
    x='Customer Name',
    y='Order Profit Per Order',
    color='Customer Segment',
    title='Top 20 Customers by Profit',
    labels={'Order Profit Per Order': 'Total Profit', 'Customer Name': 'Customer'},
    hover_data=['Customer Country', 'Customer City']
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# Customer segmentation analysis
segment_analysis = df.groupby('Customer Segment').agg({
    'Order Id': 'nunique',
    'Order Item Quantity': 'sum',
    'Order Profit Per Order': 'sum',
    'Sales': 'sum'
}).reset_index()

print("\nCustomer Segment Analysis:")
print(segment_analysis)

# Create an interactive grouped bar chart for segment analysis
fig = px.bar(
    segment_analysis,
    x='Customer Segment',
    y=['Order Id', 'Order Item Quantity', 'Order Profit Per Order', 'Sales'],
    title='Analysis by Customer Segment',
    barmode='group',
    labels={'value': 'Value', 'variable': 'Metric'}
)
fig.show()




# Customer Analysis

Top 20 Customers by Order Quantity:
          Customer Name Customer Segment Customer Country  Order Item Quantity
4958   Dorothy Friedman      Home Office          EE. UU.                  114
1636     Betty Phillips         Consumer      Puerto Rico                  111
12182        Mary Smith         Consumer          EE. UU.                  110
5603        Jerry Smith      Home Office      Puerto Rico                  103
10973       Aaron Smith         Consumer          EE. UU.                  101
5998     Rebecca Arnold         Consumer          EE. UU.                  101
6231        Mary Garcia         Consumer          EE. UU.                  100
10505  Deborah Humphrey        Corporate          EE. UU.                   99
5664        Kelly Smith        Corporate      Puerto Rico                   99
1896   Stephanie Archer      Home Office      Puerto Rico                   95
4209        Mary Butler         Consumer      Puerto Rico               


Top 20 Customers by Profit:
              Customer Name Customer Segment Customer Country  \
2612           Betty Spears         Consumer          EE. UU.   
1636         Betty Phillips         Consumer      Puerto Rico   
9755            Jacob Smith         Consumer      Puerto Rico   
2597            Laura Smith         Consumer          EE. UU.   
4958       Dorothy Friedman      Home Office          EE. UU.   
3697             Mary Lewis        Corporate          EE. UU.   
741          Jesse Matthews         Consumer      Puerto Rico   
5509         Mary Rodriguez        Corporate      Puerto Rico   
10875  Alexander Cunningham         Consumer      Puerto Rico   
5007             Mary Smith        Corporate      Puerto Rico   
2198            Ann Mercado        Corporate      Puerto Rico   
5129          Mary Sullivan        Corporate          EE. UU.   
12124        Nicholas Smith         Consumer          EE. UU.   
1133          Mary Anderson         Consumer          EE. UU.


Customer Segment Analysis:
  Customer Segment  Order Id  Order Item Quantity  Order Profit Per Order  \
0         Consumer     34119               199234              2073487.67   
1        Corporate     19856               116560              1202574.96   
2      Home Office     11777                68285               690840.34   

        Sales  
0 19095790.16  
1 11168406.84  
2  6520538.02  


In [19]:
print("\n\n# Product Analysis")
print("==================")

# Group by category and calculate number of orders
category_orders = df.groupby('Category Name').agg({
    'Order Id': 'nunique',
    'Order Item Quantity': 'sum',
    'Sales': 'sum',
    'Order Profit Per Order': 'sum'
}).reset_index()

category_orders = category_orders.sort_values('Order Id', ascending=False)

print("\nProduct Category Analysis:")
print(category_orders)

# Create an interactive bar chart for category orders
fig = px.bar(
    category_orders,
    x='Category Name',
    y='Order Id',
    title='Number of Orders by Product Category',
    labels={'Order Id': 'Number of Orders', 'Category Name': 'Product Category'},
    color='Sales',
    color_continuous_scale='Viridis',
    hover_data=['Order Item Quantity', 'Order Profit Per Order']
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# Top products by sales
product_sales = df.groupby('Product Name').agg({
    'Sales': 'sum',
    'Order Item Quantity': 'sum',
    'Category Name': 'first'
}).reset_index()

top_products = product_sales.sort_values('Sales', ascending=False).head(20)
print("\nTop 20 Products by Sales:")
print(top_products)

# Create an interactive bar chart for top products
fig = px.bar(
    top_products,
    x='Product Name',
    y='Sales',
    color='Category Name',
    title='Top 20 Products by Sales',
    labels={'Sales': 'Total Sales', 'Product Name': 'Product'},
    hover_data=['Order Item Quantity']
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# 5. Geographic Analysis
# =====================

print("\n\n# Geographic Analysis")
print("=====================")

# Orders by region
region_orders = df.groupby('Order Region').agg({
    'Order Id': 'nunique',
    'Sales': 'sum',
    'Order Profit Per Order': 'sum'
}).reset_index()

print("\nOrders by Region:")
print(region_orders)

# Create an interactive bar chart for orders by region
fig = px.bar(
    region_orders,
    x='Order Region',
    y='Order Id',
    title='Number of Orders by Region',
    labels={'Order Id': 'Number of Orders', 'Order Region': 'Region'},
    color='Sales',
    color_continuous_scale='Viridis',
    hover_data=['Order Profit Per Order']
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# Orders by country
country_orders = df.groupby('Order Country').agg({
    'Order Id': 'nunique',
    'Sales': 'sum',
    'Order Profit Per Order': 'sum'
}).reset_index()

country_orders = country_orders.sort_values('Order Id', ascending=False)
print("\nTop Countries by Number of Orders:")
print(country_orders.head(10))



# Product Analysis

Product Category Analysis:
           Category Name  Order Id  Order Item Quantity      Sales  \
12                Cleats     20386                73734 4431942.78   
34        Men's Footwear     18783                22246 2891757.66   
47       Women's Apparel     17869                62956 3147800.00   
30  Indoor/Outdoor Games     16623                57803 2888993.91   
18               Fishing     15164                17325 6929653.69   
46          Water Sports     13758                15540 3113844.68   
9       Camping & Hiking     12299                13729 4118425.57   
10      Cardio Equipment     11355                37587 3694843.20   
38         Shop By Sport     10136                32726 1309522.04   
17           Electronics      3061                 9436  371034.64   
0            Accessories      1753                 5349  133671.51   
24            Golf Balls      1444                 4384   77098.16   
21        Girls' Apparel      1191       


Top 20 Products by Sales:
                                      Product Name      Sales  \
24       Field & Stream Sportsman 16 Gun Fire Safe 6929653.69   
71                Perfect Fitness Perfect Rip Deck 4421143.14   
21   Diamondback Women's Serene Classic Comfort Bi 4118425.57   
61               Nike Men's Free 5.0+ Running Shoe 3667633.20   
59            Nike Men's Dri-FIT Victory Golf Polo 3147800.00   
70                     Pelican Sunstream 100 Kayak 3099845.09   
56         Nike Men's CJ Elite 2 TD Football Cleat 2891757.66   
67                O'Brien Men's Neoprene Life Vest 2888993.91   
102  Under Armour Girls' Toddler Spine Surge Runni 1269082.67   
18                                     Dell Laptop  663000.00   
110                                     Web Camera  267607.69   
46                                      Lawn mower  257768.73   
11                              Children's heaters  232829.20   
74                                Porcelain crafts  223356.33  



# Geographic Analysis

Orders by Region:
       Order Region  Order Id      Sales  Order Profit Per Order
0            Canada       309  186861.04                23900.71
1         Caribbean      2806 1651019.33               171825.64
2    Central Africa       556  327263.02                33447.27
3   Central America      9396 5665712.10               616341.57
4      Central Asia       184  109839.93                13045.28
5       East Africa       613  376234.90                43167.73
6       East of USA      2323 1371111.99               156263.30
7      Eastern Asia      3318 1486401.34               147368.01
8    Eastern Europe      1292  774266.57                79717.05
9      North Africa      1064  634752.22                64599.86
10  Northern Europe      3716 2155830.65               233450.60
11          Oceania      4362 2016654.20               201478.02
12    South America      4979 2960881.41               335154.40
13       South Asia      3335 1553680.92       


Top Countries by Number of Orders:
      Order Country  Order Id      Sales  Order Profit Per Order
48   Estados Unidos      8270 4879667.67               540413.07
53          Francia      4866 2879942.36               327828.58
102          México      4395 2633195.29               303278.37
8         Australia      3798 1694621.67               170041.58
2          Alemania      3518 2074171.82               194827.08
120     Reino Unido      2785 1612094.85               180942.88
20           Brasil      2650 1594319.95               186713.64
31            China      2616 1172902.11               122190.92
69            India      2152  962396.70                99746.82
70        Indonesia      1920  847605.93                90714.08


In [20]:
# Create an interactive bar chart for orders by country
fig = px.bar(
    country_orders.head(15),
    x='Order Country',
    y='Order Id',
    title='Number of Orders by Country (Top 15)',
    labels={'Order Id': 'Number of Orders', 'Order Country': 'Country'},
    color='Sales',
    color_continuous_scale='Viridis',
    hover_data=['Order Profit Per Order']
)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

# Create a choropleth map for orders by country
fig = px.choropleth(
    country_orders,
    locations='Order Country',
    locationmode='country names',
    color='Sales',
    hover_name='Order Country',
    title='Global Distribution of Orders',
    color_continuous_scale='Viridis',
    hover_data=['Sales', 'Order Profit Per Order']
)
fig.show()

# Create a scatter geo map for orders with latitude and longitude
# Filter out rows with missing lat/long
geo_df = df.dropna(subset=['Latitude', 'Longitude'])
geo_df = geo_df.groupby(['Customer City', 'Customer Country', 'Latitude', 'Longitude']).agg({
    'Order Id': 'nunique',
    'Sales': 'sum'
}).reset_index()

fig = px.scatter_geo(
    geo_df,
    lat='Latitude',
    lon='Longitude',
    color='Sales',
    size='Order Id',
    hover_name='Customer City',
    hover_data=['Customer Country', 'Sales'],
    title='Geographic Distribution of Orders',
    projection='natural earth'
)
fig.show()

# 6. Sales Analysis
# ================

print("\n\n# Sales Analysis")
print("================")

# Sales by payment type (Order Type)
payment_sales = df.groupby('Type').agg({
    'Sales': 'sum',
    'Order Id': 'nunique',
    'Order Profit Per Order': 'sum'
}).reset_index()

print("\nSales by Payment Type:")
print(payment_sales)

# Create an interactive bar chart for sales by payment type
fig = px.bar(
    payment_sales,
    x='Type',
    y='Sales',
    title='Sales by Payment Type',
    labels={'Sales': 'Total Sales', 'Type': 'Payment Type'},
    color='Order Profit Per Order',
    color_continuous_scale='Viridis',
    hover_data=['Order Id']
)
fig.show()

# Sales by delivery status
delivery_sales = df.groupby('Delivery Status').agg({
    'Sales': 'sum',
    'Order Id': 'nunique',
    'Order Profit Per Order': 'sum'
}).reset_index()

print("\nSales by Delivery Status:")
print(delivery_sales)

# Create an interactive bar chart for sales by delivery status
fig = px.bar(
    delivery_sales,
    x='Delivery Status',
    y='Sales',
    title='Sales by Delivery Status',
    labels={'Sales': 'Total Sales', 'Delivery Status': 'Delivery Status'},
    color='Order Profit Per Order',
    color_continuous_scale='Viridis',
    hover_data=['Order Id']
)
fig.show()

# Sales by shipping mode
shipping_sales = df.groupby('Shipping Mode').agg({
    'Sales': 'sum',
    'Order Id': 'nunique',
    'Order Profit Per Order': 'sum'
}).reset_index()

print("\nSales by Shipping Mode:")
print(shipping_sales)

# Create an interactive bar chart for sales by shipping mode
fig = px.bar(
    shipping_sales,
    x='Shipping Mode',
    y='Sales',
    title='Sales by Shipping Mode',
    labels={'Sales': 'Total Sales', 'Shipping Mode': 'Shipping Mode'},
    color='Order Profit Per Order',
    color_continuous_scale='Viridis',
    hover_data=['Order Id']
)
fig.show()



# Sales Analysis

Sales by Payment Type:
       Type       Sales  Order Id  Order Profit Per Order
0      CASH  4022624.17      7249               457981.09
1     DEBIT 14076857.66     25340              1530648.79
2   PAYMENT  8490351.21     15086               887239.03
3  TRANSFER 10194901.98     18077              1091034.06



Sales by Delivery Status:
     Delivery Status       Sales  Order Id  Order Profit Per Order
0   Advance shipping  8518007.89     15127               935225.28
1      Late delivery 20126395.27     36048              2140051.68
2  Shipping canceled  1570305.36      2855               160482.34
3   Shipping on time  6570026.50     11722               731143.67



Sales by Shipping Mode:
    Shipping Mode       Sales  Order Id  Order Profit Per Order
0     First Class  5674369.76     10079               643121.92
1        Same Day  1942528.56      3571               203018.43
2    Second Class  7145444.82     12778               750308.17
3  Standard Class 22022391.88     39324              2370454.45


In [21]:
# 7. Time-based Analysis
# =====================

print("\n\n# Time-based Analysis")
print("=====================")

# Extract date components directly from the datetime column
df['Order Year'] = df['order date (DateOrders)'].dt.year
df['Order Month'] = df['order date (DateOrders)'].dt.month
df['Order Day'] = df['order date (DateOrders)'].dt.day
df['Order Weekday'] = df['order date (DateOrders)'].dt.day_name()

# Monthly sales trend - Fixed approach
# Group by year and month explicitly named
monthly_sales = df.groupby([df['order date (DateOrders)'].dt.year.rename('Year'), df['order date (DateOrders)'].dt.month.rename('Month')]).agg({
    'Sales': 'sum',
    'Order Id': 'nunique',
    'Order Profit Per Order': 'sum'
}).reset_index()

# The rename columns step is now redundant as the index levels were already named
monthly_sales.columns = ['Year', 'Month', 'Sales', 'Order Count', 'Profit']

# Create a proper date column for plotting
monthly_sales['Date'] = pd.to_datetime(monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month'].astype(str) + '-01')
monthly_sales = monthly_sales.sort_values('Date')

print("\nMonthly Sales Trend:")
print(monthly_sales.head())

# Create an interactive line chart for monthly sales
fig = px.line(
    monthly_sales,
    x='Date',
    y='Sales',
    title='Monthly Sales Trend',
    labels={'Sales': 'Total Sales', 'Date': 'Month'},
    markers=True,
    hover_data=['Order Count', 'Profit']
)
fig.show()

# Sales by day of week
weekday_sales = df.groupby('Order Weekday').agg({
    'Sales': 'sum',
    'Order Id': 'nunique',
    'Order Profit Per Order': 'sum'
}).reset_index()

# Define weekday order for sorting
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_sales['Order Weekday'] = pd.Categorical(weekday_sales['Order Weekday'], categories=weekday_order, ordered=True)
weekday_sales = weekday_sales.sort_values('Order Weekday')

print("\nSales by Day of Week:")
print(weekday_sales)

# Create an interactive bar chart for sales by day of week
fig = px.bar(
    weekday_sales,
    x='Order Weekday',
    y='Sales',
    title='Sales by Day of Week',
    labels={'Sales': 'Total Sales', 'Order Weekday': 'Day of Week'},
    color='Order Profit Per Order',
    color_continuous_scale='Viridis',
    hover_data=['Order Profit Per Order']
)
fig.show()



# Time-based Analysis

Monthly Sales Trend:
   Year  Month      Sales  Order Count    Profit       Date
0  2015      1 1051590.08         1787 111660.74 2015-01-01
1  2015      2  927009.90         1585  99140.66 2015-02-01
2  2015      3 1051253.69         1781 113778.21 2015-03-01
3  2015      4 1014463.28         1710 108083.68 2015-04-01
4  2015      5 1050478.44         1776 112147.90 2015-05-01



Sales by Day of Week:
  Order Weekday      Sales  Order Id  Order Profit Per Order
1        Monday 5280297.54      9368               612331.43
5       Tuesday 5195252.04      9395               544637.39
6     Wednesday 5088116.31      9397               544067.98
4      Thursday 5237197.70      9395               520938.49
0        Friday 5306701.08      9400               564628.25
2      Saturday 5376950.24      9430               596359.61
3        Sunday 5300220.10      9367               583939.82


In [22]:
# 8. Predictive Modeling for Late Delivery Risk
# ============================================

print("\n\n# Predictive Modeling for Late Delivery Risk")
print("============================================")

# Prepare the data for modeling
# Select relevant features for prediction
features = [
    'Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
    'Shipping Mode', 'Order Region', 'Customer City', 'Category Name',
    'Order Item Quantity', 'Shipping Delay'
]

# Create a copy of the dataframe with selected features
model_df = df[features + ['Late_delivery_risk']].copy()

# Check for missing values in the model dataframe
print("\nMissing values in model dataframe:")
print(model_df.isnull().sum())

# Drop rows with missing values
model_df = model_df.dropna()
print(f"\nShape after dropping missing values: {model_df.shape}")

# Split the data into features and target
X = model_df.drop('Late_delivery_risk', axis=1)
y = model_df['Late_delivery_risk']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# Define categorical and numerical features
categorical_features = ['Type', 'Shipping Mode', 'Order Region', 'Customer City', 'Category Name']
numerical_features = ['Days for shipping (real)', 'Days for shipment (scheduled)', 'Order Item Quantity', 'Shipping Delay']

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Create and train the model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import roc_auc_score

print("\nModel Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# Calculate ROC-AUC score
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of the positive class
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Create a heatmap for the confusion matrix
fig = px.imshow(
    cm,
    labels=dict(x="Predicted", y="Actual", color="Count"),
    x=['Not Late', 'Late'],
    y=['Not Late', 'Late'],
    title='Confusion Matrix',
    color_continuous_scale='Viridis',
    text_auto=True
)
fig.show()

# Feature importance
if hasattr(model['classifier'], 'feature_importances_'):
    # Get feature names after one-hot encoding
    ohe = model['preprocessor'].named_transformers_['cat']
    cat_features = ohe.get_feature_names_out(categorical_features)
    feature_names = list(numerical_features) + list(cat_features)

    # Get feature importances
    importances = model['classifier'].feature_importances_

    # Create a dataframe for feature importances
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })

    # Sort by importance
    feature_importance = feature_importance.sort_values('Importance', ascending=False)

    print("\nFeature Importance:")
    print(feature_importance.head(10))

    # Create an interactive bar chart for feature importance
    fig = px.bar(
        feature_importance.head(15),
        x='Importance',
        y='Feature',
        title='Top 15 Feature Importances',
        orientation='h',
        color='Importance',
        color_continuous_scale='Viridis'
    )
    fig.show()

# 9. Summary and Conclusions
# =========================

print("\n\n# Summary and Conclusions")
print("=========================")

print("""
Key Findings:
1. Customer Segmentation: The dataset contains customers from different segments (Consumer, Corporate, Home Office).
2. Order Status: Most orders are in COMPLETE or PENDING_PAYMENT status.
3. Shipping Modes: Second Class is the most common shipping mode.
4. Delivery Status: A significant portion of deliveries are late.
5. Geographic Distribution: Orders are distributed across different regions and countries.
6. Time Trends: Sales show seasonal patterns throughout the year.
7. Late Delivery Prediction: Our model can predict late deliveries with reasonable accuracy.

Recommendations:
1. Focus on improving delivery times for Second Class shipping.
2. Investigate the causes of late deliveries in specific regions.
3. Optimize inventory for top-selling product categories.
4. Develop targeted marketing strategies for different customer segments.
5. Implement the predictive model to identify orders at risk of late delivery.
""")

print("\nAnalysis Complete!")



# Predictive Modeling for Late Delivery Risk

Missing values in model dataframe:
Type                             0
Days for shipping (real)         0
Days for shipment (scheduled)    0
Shipping Mode                    0
Order Region                     0
Customer City                    0
Category Name                    0
Order Item Quantity              0
Shipping Delay                   0
Late_delivery_risk               0
dtype: int64

Shape after dropping missing values: (180519, 10)

Training set shape: (144415, 9)
Testing set shape: (36104, 9)

Model Evaluation:
Accuracy: 0.9785
ROC-AUC Score: 0.9956

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98     16307
           1       0.97      0.99      0.98     19797

    accuracy                           0.98     36104
   macro avg       0.98      0.98      0.98     36104
weighted avg       0.98      0.98      0.98     36104


Confusion Matrix:
[[15648   65


Feature Importance:
                          Feature  Importance
3                  Shipping Delay        0.49
0        Days for shipping (real)        0.25
11   Shipping Mode_Standard Class        0.06
8       Shipping Mode_First Class        0.05
1   Days for shipment (scheduled)        0.04
10     Shipping Mode_Second Class        0.02
2             Order Item Quantity        0.01
7                   Type_TRANSFER        0.01
9          Shipping Mode_Same Day        0.00
5                      Type_DEBIT        0.00




# Summary and Conclusions

Key Findings:
1. Customer Segmentation: The dataset contains customers from different segments (Consumer, Corporate, Home Office).
2. Order Status: Most orders are in COMPLETE or PENDING_PAYMENT status.
3. Shipping Modes: Second Class is the most common shipping mode.
4. Delivery Status: A significant portion of deliveries are late.
5. Geographic Distribution: Orders are distributed across different regions and countries.
6. Time Trends: Sales show seasonal patterns throughout the year.
7. Late Delivery Prediction: Our model can predict late deliveries with reasonable accuracy.

Recommendations:
1. Focus on improving delivery times for Second Class shipping.
2. Investigate the causes of late deliveries in specific regions.
3. Optimize inventory for top-selling product categories.
4. Develop targeted marketing strategies for different customer segments.
5. Implement the predictive model to identify orders at risk of late delivery.


Analysis Complete!
